In [111]:
import pandas as pd
df=pd.read_csv('new_data.csv')
df=df[(df.type_char!='file_folder')&(df.type_char!='handwritten')]
df.head()

,Unnamed: 0,filename,type,text_image,type_char
0,0,img_0000000.jpg,0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,facture
1,1,img_0000001.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture
2,2,img_0000002.jpg,0,FACTURE\n\nMon entreprise : Nom de la société\...,facture
3,3,img_0000003.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture
4,4,img_0000004.jpg,0,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,facture


In [112]:
#nettoyage de text 
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
stop_words.update(["." , ",",':','?'])
stop_words=list(stop_words)
stop_words.extend(stopwords.words('french'))
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

In [113]:
df.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))
df.head()

,Unnamed: 0,filename,type,text_image,type_char
0,0,img_0000000.jpg,0,facture logo joanna binet coubertin paris fact...,facture
1,1,img_0000001.jpg,0,joanna binet coubertin paris facture facture c...,facture
2,2,img_0000002.jpg,0,facture entreprise nom societe adresse postale...,facture
3,3,img_0000003.jpg,0,joanna binet coubertin paris facture cendrillo...,facture
4,4,img_0000004.jpg,0,payer ligne facture sfideli date creation date...,facture


In [117]:
#diviser le jeu de données 
from sklearn.model_selection import train_test_split
X_text_train,X_text_test, y_train, y_test=train_test_split(df.text_image,df.type,test_size=0.2,random_state=1234)

In [118]:
#tokenisation
import  tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_text_train)
X_train=tokenizer.texts_to_sequences(X_text_train)
X_test=tokenizer.texts_to_sequences(X_text_test)
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=100, padding='post', truncating='post')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=100, padding='post', truncating='post')

In [122]:
#créer un reseaux de neuron
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout
model=Sequential()
model.add(Embedding(10000, 200))
model.add(GlobalAveragePooling1D())
model.add(Dense(units = 256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(units = 23, activation = "softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 200)         2000000   
                                                                 
 global_average_pooling1d_1   (None, 200)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 256)               51456     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 23)                5911      
                                                                 
Total params: 2,057,367
Trainable params: 2,057,367
Non-trainable params: 0
____________________________________________

In [123]:
#entrainement
model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train,y_train,epochs = 50, batch_size = 32,validation_data = [X_test, y_test])

Epoch 1/50
39/39 [==============================] - 2s 27ms/step - loss: 2.8937 - accuracy: 0.2680 - val_loss: 2.6088 - val_accuracy: 0.2848
Epoch 2/50
39/39 [==============================] - 1s 24ms/step - loss: 2.4194 - accuracy: 0.3522 - val_loss: 2.3094 - val_accuracy: 0.3981
Epoch 3/50
39/39 [==============================] - 1s 23ms/step - loss: 2.0258 - accuracy: 0.4081 - val_loss: 2.0703 - val_accuracy: 0.4142
Epoch 4/50
39/39 [==============================] - 1s 23ms/step - loss: 1.7493 - accuracy: 0.4640 - val_loss: 1.9107 - val_accuracy: 0.4498
Epoch 5/50
39/39 [==============================] - 1s 23ms/step - loss: 1.5379 - accuracy: 0.5644 - val_loss: 1.7796 - val_accuracy: 0.4790
Epoch 6/50
39/39 [==============================] - 1s 24ms/step - loss: 1.3796 - accuracy: 0.6138 - val_loss: 1.6794 - val_accuracy: 0.5016
Epoch 7/50
39/39 [==============================] - 1s 24ms/step - loss: 1.2587 - accuracy: 0.6397 - val_loss: 1.5959 - val_accuracy: 0.5178
Epoch 8/50
39

In [121]:
#evaluation 
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 3ms/step - loss: 1.3480 - accuracy: 0.6537


[1.3480002880096436, 0.6537216901779175]